# Guide Me Through the Q!

![Q](images/q.png)

## Abstract

Text

##  Table of Contents

* [Part 1: Creating the Data Set](#Part-1:-Creating-the-Data-Set)
    * [Concatenating the JSON Files](#Concatenating-the-JSON-Files)
    * [Producing the Dataframe](#Producing-the-Dataframe)
    * [Cleaning](#Cleaning)
    * [Adding Statistics and Indicators](#Adding-Statistics-and-Indicators)

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

## Part 1: Creating the Data Set

### Concatenating the JSON Files

The Q data we received consisted of about 2300 JSON files, each one representing a unique course's historical Q scores for each available semester. Out first task was to combine these individual JSON files into a single JSON Lines file; the JSON files were stored locally outside of the repository directory, and we used the `shututil` and `glob` libraries to concatenate these into `qall.jsonl`.

In [2]:
import shutil
import glob

# Opens the JSONL file as output.
with open("qall.jsonl", "wb") as outfile:
    
    # Iterates over the JSON files.
    for filename in glob.glob("../Q/*.json"):
        
        # Opens the current JSON file as input.
        with open(filename, "rb") as readfile:
            
            # Copies the JSON file to the JSONL file.
            shutil.copyfileobj(readfile, outfile)
            
            # Adds a new line as a delimiter.
            outfile.write("\n")

### Producing the Dataframe

Each JSON file was structured using multiple nested dictionaries, with the following format:

* `scores`: a dictionary of mostly numerical data, separated by semester
    * `*semester*`: a dictionary of course and instructor scores
        * `course_score`: a dictionary of course info, ratings, and rating breakdowns
            * `overall`: a float of mean overall rating
            * `workload`: a float of mean workload rating
            * `difficulty`: a float of mean difficulty rating
            * `recommendation`: a float of mean recommendation rating
            * `enrollment`: an integer of class enrollment
            * `reponse rate`: a float of the proportion of students who reviewed the course
            * `QCourseOverall`: 
                * `number`: the string concatenation of department and course number, i.e. "COMPSCI 109"
                * `course_id`: an integer identifier for my.harvard.edu
                * `cat_num`: an integer indentifier for the Harvard Course Catalog
                * `1s`: an integer count of reviews with rating 1 for this category
                * `2s`: analagous, with 2s
                * `3s`: analagous, with 3s
                * `4s`: analagous, with 4s
                * `5s`: analagous, with 5s
            * `QDifficulty`: analagous to `QCourseOverall`, with difficulty ratings.
            * `QWorkload`: analagous to `QCourseOverall`, with workload ratings.
        * `instructor_scores`: a list of length 1 that contains a dictionary of instructor info and ratings
            * `number`: identical to that within `QCourseOverall`
            * `cat_num`: identical to that within `QCourseOverall`
            * `course_id`: identical to that within `QCourseOverall`
            * `year`: an int of the first of the two years represented in the given school year
            * `term`: an int with value 1 if the course took place in the Fall, 2 if Spring
            * `id`: a string indentifier of the professor
            * `first`: a string of the first name of the professor
            * `last`: a string of the last name
            * `InstructorOverall`: a float of mean overall instructor rating
            * `EffectiveLecturesorPresentations`: a float of mean lecture rating
            * `AccessibleOutsideClass`: a float of mean accessibility rating
            * `GeneratesEnthusiasm`: a float of mean enthusiasm rating
            * `FacilitatesDiscussionEncouragesParticipation`: a float of mean facilitation rating
            * `GivesUsefulFeedback`: a float of mean feedback rating
            * `ReturnsAssignmentsinTimelyFashion`: a float of mean timeliness rating
* `comments`: a dictionary of textual reviews, separated by semester into dictionaries
    * `*semester*`: a dictionary of comments for the given semester
        * `comments`: a list of strings that represent comments
* `mostRecentQ`: identical to the most recent semester data in `scores`
* `success`: indicates if the JSON was successfully retrieved

Any mean rating has range `1.0-5.0`, and response rate has a range `0.0-1.0`. The entries labeled `*semester*` can occur any number of times and have as a key the semester in which it took place (i.e. `Fall '12`).

We wanted our first dataframe `bigdf` to have a single row for each instance of a course; for example `COMPSCI 109` in `Fall '13` and `COMPSCI 109` in `Fall '14` would have separate rows. This versatile dataframe could be adjusted in a number of different ways to suit the purposes of our data analysis. One caveat was a lack of course or instructor data, or both! To handle for this, we first created an empty dataframe with specified columns; then, we loaded the data for each cousrse using the `json` library, created a minimal dictionary for each semester, added further semester data to the dictionary conditional upon the data's existence, then appended that dictionary to the dataframe.

In [3]:
import json

# Creates the dataframe with columns specified.
bigdf = pd.DataFrame(columns=["C_Department","C_Number", "Course", "C_CatNum","C_ID","C_Semester","C_Year","C_Term",
                              "C_Overall","C_Workload","C_Difficulty","C_Recommendation","C_Enrollment","C_ResponseRate",
                              "I_First","I_Last","I_ID",
                              "I_Overall","I_EffectiveLectures","I_Accessible","I_GeneratesEnthusiasm",
                              "I_EncouragesParticipation","I_UsefulFeedback","I_ReturnsAssignmentsTimely",
                              "QOverall_1","QOverall_2","QOverall_3","QOverall_4","QOverall_5",
                              "QDifficulty_1","QDifficulty_2","QDifficulty_3","QDifficulty_4","QDifficulty_5",
                              "QWorkload_1","QWorkload_2","QWorkload_3","QWorkload_4","QWorkload_5",
                              "Comments"])

# Opens the JSONL file as the input.
with open("qall.jsonl") as data_file:

    # Iterates over each line.
    for line in data_file:

        # Loads the JSON.
        alldata = json.loads(line)

        # Iterates over each semester for the course.
        for semester in alldata["scores"]:

            # Retrieves the semester data.
            semdata = alldata["scores"][semester]

            # Retrieves the scores or comments if they exist, else None.
            cscores = semdata["course_score"] if "course_score" in semdata else None
            iscores = semdata["instructor_scores"][0] if "instructor_scores" in semdata else None
            comments = alldata["comments"][semester]["comments"] if semester in alldata["comments"] else None

            # Creates a dictionary that will be appended as a row to the dataframe.
            semdatadict = {"C_Semester": semester, "Comments": comments}

            # If the course scores exist...
            if cscores != None:
                
                # Retrieves the course name and separates it into department and number.
                course = str(cscores["QCourseOverall"]["number"]).split()
                if course[0] == "None":
                    course = [None, None]
                    fullname = "NODEPT-NOCOURSENUMBER"
                else:
                    fullname = course[0] + "-" + course[1]
                    
                # Updates the dictionary with the data.
                semdatadict.update({"C_Department": course[0],
                                    "C_Number": course[1],
                                    "Course": fullname,
                                    "C_CatNum": cscores["QCourseOverall"]["cat_num"],
                                    "C_ID": cscores["QCourseOverall"]["course_id"],
                                    "C_Overall": cscores["overall"],
                                    "C_Workload": cscores["workload"],
                                    "C_Difficulty": cscores["difficulty"],
                                    "C_Recommendation": cscores["recommendation"],
                                    "C_Enrollment": cscores["enrollment"],
                                    "C_ResponseRate": cscores["response rate"],
                                    "QOverall_1": cscores["QCourseOverall"]["1s"],
                                    "QOverall_2": cscores["QCourseOverall"]["2s"],
                                    "QOverall_3": cscores["QCourseOverall"]["3s"],
                                    "QOverall_4": cscores["QCourseOverall"]["4s"],
                                    "QOverall_5": cscores["QCourseOverall"]["5s"],
                                    "QDifficulty_1": cscores["QDifficulty"]["1s"],
                                    "QDifficulty_2": cscores["QDifficulty"]["2s"],
                                    "QDifficulty_3": cscores["QDifficulty"]["3s"],
                                    "QDifficulty_4": cscores["QDifficulty"]["4s"],
                                    "QDifficulty_5": cscores["QDifficulty"]["5s"],
                                    "QWorkload_1": cscores["QWorkload"]["1s"],
                                    "QWorkload_2": cscores["QWorkload"]["2s"],
                                    "QWorkload_3": cscores["QWorkload"]["3s"],
                                    "QWorkload_4": cscores["QWorkload"]["4s"],
                                    "QWorkload_5": cscores["QWorkload"]["5s"],})

            # If the instructor scores exist...
            if iscores != None:

                # Retrieves the course name and separates it into department and number.
                course = str(iscores["number"]).split()
                if course[0] == "None":
                    course = [None, None]
                    fullname = "NODEPT-NOCOURSENUMBER"
                else:
                    fullname = course[0] + "-" + course[1]

                # Updates the dictionary with the data.
                semdatadict.update({"C_Department": course[0],
                                    "C_Number": course[1],
                                    "Course": fullname,
                                    "C_CatNum": iscores["cat_num"],
                                    "C_ID": iscores["course_id"],
                                    "C_Year": iscores["year"],
                                    "C_Term": iscores["term"],
                                    "I_First": iscores["first"].strip(),
                                    "I_Last": iscores["last"].strip(),
                                    "I_ID": iscores["id"],
                                    "I_Overall": iscores["InstructorOverall"],
                                    "I_EffectiveLectures": iscores["EffectiveLecturesorPresentations"],
                                    "I_Accessible": iscores["AccessibleOutsideClass"],
                                    "I_GeneratesEnthusiasm": iscores["GeneratesEnthusiasm"],
                                    "I_EncouragesParticipation": iscores["FacilitatesDiscussionEncouragesParticipation"],
                                    "I_UsefulFeedback": iscores["GivesUsefulFeedback"],
                                    "I_ReturnsAssignmentsTimely": iscores["ReturnsAssignmentsinTimelyFashion"]})

            # Adds the dictionary as a row to the dataframe.
            bigdf = bigdf.append(semdatadict, ignore_index=True)

Here is a look at the resulting dataframe:

In [4]:
bigdf.head(5)

,C_Department,C_Number,Course,C_CatNum,C_ID,C_Semester,C_Year,C_Term,C_Overall,C_Workload,C_Difficulty,C_Recommendation,C_Enrollment,C_ResponseRate,I_First,I_Last,I_ID,I_Overall,I_EffectiveLectures,I_Accessible,I_GeneratesEnthusiasm,I_EncouragesParticipation,I_UsefulFeedback,I_ReturnsAssignmentsTimely,QOverall_1,QOverall_2,QOverall_3,QOverall_4,QOverall_5,QDifficulty_1,QDifficulty_2,QDifficulty_3,QDifficulty_4,QDifficulty_5,QWorkload_1,QWorkload_2,QWorkload_3,QWorkload_4,QWorkload_5,Comments
0,HISTSCI,270,HISTSCI-270,58523,2697,Spring '12,2011,2,4.67,2.33,3.33,5.00,6,50.00,Rebecca,Lemov,79de794d3e2e19eb71a2033b0ec0b76d,4.67,4.33,4.00,4.33,5.00,4.50,4.00,0,0,0,1,2,0,0,2,1,0,0,2,1,0,0,[This course is a perfect example of what grea...
1,EXPOS,20.132,EXPOS-20.132,22108,1676,Fall '14,2014,1,4.10,7.10,None,3.50,13,76.92,Owen,Chen,1341ccb7bd27f47e68625b63b15281d1,4.50,4.60,3.70,3.90,3.90,4.10,4.60,0,1,1,4,4,0,1,2,7,3,1,3,4,1,0,"[The class has a fairly high work load, but Dr..."
2,EXPOS,20.132,EXPOS-20.132,22108,1676,Fall '13,2013,1,3.50,2.60,3.90,3.20,13,100.00,Owen,Chen,1341ccb7bd27f47e68625b63b15281d1,3.80,4.00,2.50,3.50,4.10,4.30,4.40,0,1,1,4,4,0,1,2,7,3,1,3,4,1,0,[Philosophy of the State with Dr. Chen offers ...
3,EXPOS,20.132,EXPOS-20.132,22108,1676,Fall '12,2012,1,3.73,2.47,3.67,3.47,15,100.00,Owen,Chen,1341ccb7bd27f47e68625b63b15281d1,3.87,4.33,2.64,3.93,4.18,3.64,3.82,0,1,1,4,4,0,1,2,7,3,1,3,4,1,0,[This was by far my favorite course. Dr. Chen ...
4,EXPOS,20.132,EXPOS-20.132,22108,1676,Fall '11,2011,1,3.85,2.00,3.54,3.62,13,100.00,Owen,Chen,1341ccb7bd27f47e68625b63b15281d1,4.08,3.75,3.31,3.92,4.46,4.23,4.08,0,1,1,4,4,0,1,2,7,3,1,3,4,1,0,"[Be prepared to read, Discussions were great, ..."


### Cleaning

If the instructor data was missing for a course, `C_Year` and `C_Term` were left out. We iterated over each row to extract this information from `C_semester`:

In [5]:
# Iterates over each row in the dataframe
for index, row in bigdf.iterrows():

    # Retrieves the year.
    year = int(row["C_Semester"][-2:])

    # Based on semester, determines year and term.
    if row["C_Semester"][0:4] == "Fall":
        row["C_Term"] = 1
        row["C_Year"] = 2000 + year
    elif row["C_Semester"][0:6] == "Spring":
        row["C_Term"] = 2
        row["C_Year"] = 1999 + year
    else:
        continue

We checked if any row lacked a department number and thus a course number as well:

In [6]:
print bigdf[bigdf["C_Department"]!=bigdf["C_Department"]].shape[0]
print bigdf[bigdf["C_Department"]==None].shape[0]

1
0


We decided to delete this course that has no number. Although there is textual data in the comments, the missing number is a result of having no course and instructor scores, so there is little we can do with it numerically:

In [7]:
bigdf = bigdf[bigdf["C_Department"]==bigdf["C_Department"]]

Then, we adjusted the types of each column to prepare for data analysis. We created a dictionary `coltypes` with keys as types and values as a list of the columns that should be those types. All columns except `Comments` were categorized as either an `int`, a `float`, or a `string`:

In [8]:
coltypes = {int: ["C_CatNum","C_ID","C_Year","C_Term","C_Enrollment",
                  "QOverall_1","QOverall_2","QOverall_3","QOverall_4","QOverall_5",
                  "QDifficulty_1","QDifficulty_2","QDifficulty_3","QDifficulty_4","QDifficulty_5",
                  "QWorkload_1","QWorkload_2","QWorkload_3","QWorkload_4","QWorkload_5"],
            float: ["C_Overall","C_Workload","C_Difficulty","C_Recommendation","C_ResponseRate",
                    "I_Overall","I_EffectiveLectures","I_Accessible","I_GeneratesEnthusiasm",
                    "I_EncouragesParticipation","I_UsefulFeedback","I_ReturnsAssignmentsTimely",
                    "QOverall_1","QOverall_2","QOverall_3","QOverall_4","QOverall_5"],
            str: ["C_Number","C_Department","C_Semester","I_First","I_Last","I_ID"]}

Some of the `string` cells contained unicode that could not be written to a csv. Using the `unidecode` library, we converted these to writable strings:

In [9]:
from unidecode import unidecode

for index, row in bigdf.iterrows():
    for strcol in coltypes[str]:
        try:
            str(row[strcol])
        except UnicodeEncodeError:
            bigdf.loc[index, row[row.values==row[strcol]].index[0]] = unidecode(row[strcol])

Then, we replaced `None` values with `NaN` for floats and `-1` for ints.

In [10]:
for intcol in coltypes[int]:
    bigdf = bigdf.replace({intcol: {None: -1, "": -1}})

for floatcol in coltypes[float]:
    bigdf = bigdf.replace({floatcol: {None: np.nan}})

To finish correcting the types, we cast the entire columns as the desired type using the keys in `coltypes`:

In [11]:
for ctp in coltypes:
    for col in coltypes[ctp]:
        bigdf[col] = bigdf[col].astype(ctp)

In [ ]:
# for index, comments in bigdf["Comments"].iteritems():
#     if comments != None:
#         bigdf["Comments"].loc[index] = [unidecode(c) for c in comments]

### Adding Statistics and Indicators

In [ ]:
bigdf["Sem_Average"] = pd.Series(index=bigdf.index)

In [ ]:
for name, group in bigdf.groupby(["C_Year", "C_Term"]):
    bigdf.loc[(bigdf["C_Year"]==name[0])&(bigdf["C_Term"]==name[1]), "Sem_Average"] = np.nanmean(group["C_Overall"].values)

In [ ]:
bigdf["Positive"] = pd.Series(bigdf["C_Overall"]>=bigdf["Sem_Average"], index=bigdf.index)

In [ ]:
bigdf.head(5)

In [ ]:
bigdf.to_csv("bigdf.csv", index=False)

## References

http://www.fas.harvard.edu/~evals/  
http://www.clipartpanda.com/categories/cool-question-marks